In [1]:
import math
import pandas as pd
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras.losses import MeanSquaredLogarithmicError
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error

In [2]:
a=pd.read_csv('silver.csv')
a['date']=(pd.to_datetime(a['date'], format='%Y-%m-%d')-pd.to_datetime('1973-01-01')).dt.days
X = a.iloc[:,[0,1,2,6]].values
Y = a.iloc[:,[3]].values
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

In [3]:
def scale_datasets(X_train, X_test):
  standard_scaler = StandardScaler()
  X_train_scaled = pd.DataFrame(standard_scaler.fit_transform(X_train))
  X_test_scaled = pd.DataFrame(standard_scaler.transform(X_test))
  return X_train_scaled, X_test_scaled
X_train_scaled, X_test_scaled = scale_datasets(X_train, X_test)

In [4]:
hidden_units1 = 160
hidden_units2 = 480
hidden_units3 = 256
learning_rate = 0.01
def build_model_using_sequential():
  model = Sequential([Dense(hidden_units1, kernel_initializer='normal', activation='relu'),Dropout(0.2),
    Dense(hidden_units2, kernel_initializer='normal', activation='relu'),Dropout(0.2),
    Dense(hidden_units3, kernel_initializer='normal', activation='relu'),
    Dense(1, kernel_initializer='normal', activation='linear')])
  return model
model = build_model_using_sequential()

In [5]:
msle = MeanSquaredLogarithmicError()
model.compile(loss=msle, optimizer=Adam(learning_rate=learning_rate), metrics=[msle])
history = model.fit(X_train_scaled, Y_train, epochs=10, batch_size=64,validation_split=0.2)

38558/38558 [==============================] - 264s 7ms/step - loss: 0.0779 - mean_squared_logarithmic_error: 0.0779 - val_loss: 0.0712 - val_mean_squared_logarithmic_error: 0.0712


In [6]:
y_1 = model.predict(X_test)
print(y_1)  
y_1 = pd.DataFrame(y_1)
y_1.to_csv('RF-EQPD.csv')

24099/24099 [==============================] - 64s 3ms/step
[[0.68374497]
 [0.68374497]
 [0.68374497]
 ...
 [0.68374497]
 [0.68374497]
 [0.68374497]]


In [7]:
print("MSE: ",mean_squared_error(Y_test,y_1)," \nR2: ",r2_score(Y_test,y_1)," \nMAE: ",mean_absolute_error(Y_test,y_1))

MSE:  26.02709818867845  
R2:  -14.20033442064869  
MAE:  1.3914766810030201
